In [4]:
import os
import math
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from itertools import cycle
from sklearn.utils import shuffle
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, \
                            r2_score, mean_poisson_deviance, mean_gamma_deviance, accuracy_score 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [5]:
df_eth = pd.read_csv('data/ETH-USD.csv')
df_eth['Date'] = pd.to_datetime(df_eth['Date'])
df_eth = df_eth[['Date','Close']]
df_eth = df_eth[df_eth['Date'] > '2021-01-01']
eth_stock = df_eth.copy()

In [6]:
del df_eth['Date']
scaler = MinMaxScaler(feature_range=(0,1))
df_eth = scaler.fit_transform(np.array(df_eth).reshape(-1,1))

In [7]:
# split dataset into train and test split
training_size = int(len(df_eth)*0.8)
test_size = len(df_eth) - training_size

train_data, test_data = df_eth[0:training_size,:], df_eth[training_size:len(df_eth),:1]

In [8]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [9]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)

X_train:  (508, 15)
y_train:  (508,)
X_test:  (116, 15)


In [10]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (508, 15, 1)
X_test:  (116, 15, 1)


In [11]:
model=Sequential()
model.add(LSTM(10,input_shape=(None,1),activation="relu"))
model.add(Dense(1))
model.compile(loss="mean_squared_error",optimizer="adam")
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Epoch 1/200
16/16 [==============================] - 3s 61ms/step - loss: 0.1702 - val_loss: 0.0065
Epoch 2/200
16/16 [==============================] - 1s 45ms/step - loss: 0.0914 - val_loss: 0.0015
Epoch 3/200
16/16 [==============================] - 1s 44ms/step - loss: 0.0271 - val_loss: 0.0094
Epoch 4/200
16/16 [==============================] - 1s 44ms/step - loss: 0.0086 - val_loss: 0.0140
Epoch 5/200
16/16 [==============================] - 1s 46ms/step - loss: 0.0075 - val_loss: 0.0091
Epoch 6/200
16/16 [==============================] - 1s 54ms/step - loss: 0.0069 - val_loss: 0.0096
Epoch 7/200
16/16 [==============================] - 1s 57ms/step - loss: 0.0067 - val_loss: 0.0087
Epoch 8/200
16/16 [==============================] - 1s 55ms/step - loss: 0.0064 - val_loss: 0.0075
Epoch 9/200
16/16 [==============================] - 1s 58ms/step - loss: 0.0063 - val_loss: 0.0070
Epoch 10/200
16/16 [==============================] - 1s 58ms/step - loss: 0.0062 - val_loss: 0.0065